## Exc day 9

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/snorreralund/scraping_seminar/master/english_review_sample.csv')

In [92]:
travelling = set(['airport', 'airplane', 'bus', 'delay', 'weather'])
computers = set(['inventory', 'hardware', 'software', 'laptop', 'notebook'])

In [61]:
print('airport' in travelling)

True


In [190]:
import nltk
tokenized_docs = []
for i in df.reviewBody:
    toks = nltk.word_tokenize(i)
    words = set([w.lower() for w in toks])
    tokenized_docs.append(words)

In [95]:
overlap = []
for i in tokenized_docs:
    overlap_ = len([value for value in i if value in computers])
    overlap.append(overlap_)

In [96]:
df['overlap'] = overlap
df.head()

,Unnamed: 0,__domain__,address_@type,address_addressCountry,address_addressLocality,address_postalCode,address_streetAddress,author_@type,datePublished,email,...,itemReviewed_@type,itemReviewed_name,meta_@type,name,reviewBody,reviewRating_@type,reviewRating_ratingValue,telephone,categories,overlap
0,159770,https://trustpilot.com/review/www.exmed.net,PostalAddress,NaN,Fenton,63026,218 Seebold Spur,Person,2017-07-29T20:27:03Z,sales@exmed.net,...,Thing,Express Medical Supply,LocalBusiness,Express Medical Supply,"Lots of inventory, very fast and efficient. I ...",Rating,5,(800) 633-2139,/health_wellbeing,1
1,168724,https://trustpilot.com/review/mapscompany.com,PostalAddress,NaN,"Petit-Rocher, NB",E8J 1E4,713 rue de la Mer,Person,2017-08-11T20:09:48Z,contact@mapscompany.com,...,Thing,MapsCompany,LocalBusiness,MapsCompany,I did not received the map I had ordered and p...,Rating,3,NaN,/travel_holidays,0
2,96443,https://trustpilot.com/review/www.thriftbooks.com,PostalAddress,NaN,Tukwila,98188,"18300 Cascade Ave S, Ste 150",Person,2015-03-19T22:59:22Z,reviews@thriftbooks.com,...,Thing,Thrift Books,LocalBusiness,Thrift Books,After searching a number of stores here in my ...,Rating,5,253-275-2251,/entertainment,0
3,173433,https://trustpilot.com/review/fabletics.com,PostalAddress,NaN,NaN,NaN,NaN,Person,2017-04-30T19:47:39Z,NaN,...,Thing,Fabletics,LocalBusiness,Fabletics,Website is not intuitive. I don't like having...,Rating,2,855-202-3570,/clothes_fashion,0
4,138968,https://trustpilot.com/review/www.enterprise.com,PostalAddress,US,St Louis,63105,600 Corporate Park Dr,Person,2018-05-26T20:43:41Z,NaN,...,Thing,Enterprise,LocalBusiness,Enterprise,"Outstanding customer service, appreciated the ...",Rating,5,NaN,/transportation,0


## 17.2

In [133]:
# nltk.download('opinion_lexicon')
from nltk.corpus import opinion_lexicon

In [134]:
positive_words = opinion_lexicon.positive
negative_words = opinion_lexicon.negative

In [99]:
import pandas as pd
vader_df = pd.read_csv('https://raw.githubusercontent.com/cjhutto/vaderSentiment/master/vaderSentiment/vader_lexicon.txt'
          ,sep='\t',header=None) # changing the separator to tab, and specifying no header
vader_df.columns = ['token','average_score','variance','ratings'] # adding the header

In [100]:
vader_df.head()

,token,average_score,variance,ratings
0,$:,-1.5,0.80623,"[-1, -1, -1, -1, -3, -1, -3, -1, -2, -1]"
1,%),-0.4,1.01980,"[-1, 0, -1, 0, 0, -2, -1, 2, -1, 0]"
2,%-),-1.5,1.43178,"[-2, 0, -2, -2, -1, 2, -2, -3, -2, -3]"
3,&-:,-0.4,1.42829,"[-3, -1, 0, 0, -1, -1, -1, 2, -1, 2]"
4,&:,-0.7,0.64031,"[0, -1, -1, -1, 1, -1, -1, -1, -1, -1]"


In [191]:
negative = set(vader_df.loc[vader_df['average_score'] < 0])
positive = set(vader_df.loc[vader_df['average_score'] > 0])

In [192]:
vader_lexicon = {'positive':positive,'negative':negative}
opinion_lexicon = {'positive':positive_words,'negative':negative_words}

In [212]:
def apply_sentiment_dictionary(doc):
    pol_v = []
    for i in doc:    
        p_vader = len([value for value in i if value not in vader_lexicon['positive']])
        n_vader = len([value for value in i if value not in vader_lexicon['negative']])
        ps_vader = p_vader-n_vader/len(i)
        pol_v.append(ps_vader)
    return pol_v
    
df['vader_raw'] = apply_sentiment_dictionary(tokenized_docs)

,Unnamed: 0,__domain__,address_@type,address_addressCountry,address_addressLocality,address_postalCode,address_streetAddress,author_@type,datePublished,email,...,itemReviewed_name,meta_@type,name,reviewBody,reviewRating_@type,reviewRating_ratingValue,telephone,categories,overlap,vader_raw
0,159770,https://trustpilot.com/review/www.exmed.net,PostalAddress,NaN,Fenton,63026,218 Seebold Spur,Person,2017-07-29T20:27:03Z,sales@exmed.net,...,Express Medical Supply,LocalBusiness,Express Medical Supply,"Lots of inventory, very fast and efficient. I ...",Rating,5,(800) 633-2139,/health_wellbeing,1,13.0
1,168724,https://trustpilot.com/review/mapscompany.com,PostalAddress,NaN,"Petit-Rocher, NB",E8J 1E4,713 rue de la Mer,Person,2017-08-11T20:09:48Z,contact@mapscompany.com,...,MapsCompany,LocalBusiness,MapsCompany,I did not received the map I had ordered and p...,Rating,3,NaN,/travel_holidays,0,62.0
2,96443,https://trustpilot.com/review/www.thriftbooks.com,PostalAddress,NaN,Tukwila,98188,"18300 Cascade Ave S, Ste 150",Person,2015-03-19T22:59:22Z,reviews@thriftbooks.com,...,Thrift Books,LocalBusiness,Thrift Books,After searching a number of stores here in my ...,Rating,5,253-275-2251,/entertainment,0,47.0
3,173433,https://trustpilot.com/review/fabletics.com,PostalAddress,NaN,NaN,NaN,NaN,Person,2017-04-30T19:47:39Z,NaN,...,Fabletics,LocalBusiness,Fabletics,Website is not intuitive. I don't like having...,Rating,2,855-202-3570,/clothes_fashion,0,25.0
4,138968,https://trustpilot.com/review/www.enterprise.com,PostalAddress,US,St Louis,63105,600 Corporate Park Dr,Person,2018-05-26T20:43:41Z,NaN,...,Enterprise,LocalBusiness,Enterprise,"Outstanding customer service, appreciated the ...",Rating,5,NaN,/transportation,0,14.0
